## Run fixed combinations


In [ ]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
from agent import DQNAgent
from tqdm.auto import tqdm
import random
import itertools

room_size = "xl-different-prob"
terminates_at = 99
num_iterations = (terminates_at + 1) * 100
replay_buffer_size = num_iterations
warm_start = num_iterations // 10
batch_size = 32
target_update_interval = 10
gamma_mm = 0.99
gamma_explore = 0.9


for capacity_max in [12, 24, 48]:
    prob_type = (
        "non-equal-object-probs"
        if "different-prob" in room_size
        else "equal-object-probs"
    )
    root_path = (
        f"./training-results/{prob_type}/dqn/"
        f"room_size={room_size}/capacity={capacity_max}/"
    )
    for test_seed in [0, 1, 2, 3, 4]:

        for agent_type in ["hybrid", "semantic", "episodic"]:

            if agent_type == "hybrid":
                capacity = {
                    "episodic": capacity_max // 2,
                    "semantic": capacity_max // 2,
                    "short": 1,
                }
            elif agent_type == "episodic":
                capacity = {"episodic": capacity_max, "semantic": 0, "short": 1}
            elif agent_type == "semantic":
                capacity = {"episodic": 0, "semantic": capacity_max, "short": 1}
            else:
                raise ValueError(f"Unknown agent_type: {agent_type}")

            params_dict = {
                "env_str": "rooms_env:RoomsEnv",
                "num_iterations": num_iterations,
                "replay_buffer_size": replay_buffer_size,
                "validation_starts_at": 0,
                "warm_start": warm_start,
                "batch_size": batch_size,
                "target_update_interval": target_update_interval,
                "epsilon_decay_until": num_iterations,
                "max_epsilon": 1.0,
                "min_epsilon": 0.1,
                "gamma": {"mm": gamma_mm, "explore": gamma_explore},
                "capacity": capacity,
                "pretrain_semantic": False,
                "semantic_decay_factor": 0.9,
                "lstm_params": {
                    "num_layers": 2,
                    "embedding_dim": 64,
                    "hidden_size": 64,
                    "bidirectional": False,
                    "max_timesteps": terminates_at + 1,
                    "max_strength": terminates_at + 1,
                    "relu_for_attention": True,
                },
                "mlp_params": {
                    "hidden_size": 64,
                    "num_hidden_layers": 1,
                    "dueling_dqn": True,
                },
                "num_samples_for_results": {"val": 10, "test": 10},
                "validation_interval": 1,
                "plotting_interval": 50,
                "train_seed": test_seed + 5,
                "test_seed": test_seed,
                "device": "cpu",
                "qa_function": "episodic_semantic",
                "explore_policy_heuristic": "avoid_walls",
                "env_config": {
                    "question_prob": 1.0,
                    "terminates_at": terminates_at,
                    "randomize_observations": "objects",
                    "room_size": room_size,
                    "rewards": {"correct": 1, "wrong": 0, "partial": 0},
                    "make_everything_static": False,
                    "num_total_questions": 1000,
                    "question_interval": 1,
                    "include_walls_in_observations": True,
                },
                "ddqn": True,
                "default_root_dir": root_path,
            }

            agent = DQNAgent(**params_dict)
            agent.train()